<a href="https://colab.research.google.com/github/UOS-COMP6252/public/blob/main/WGAN/wgan_gp_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Wassertein GAN

This is a notebook to generate images from Wasserstein GAN with gradient penalty pre-trained on the celebA dataset

In [ ]:
!pip install comet_ml

In [ ]:
%%sh
wget https://github.com/UOS-COMP6252/public/raw/main/WGAN/dcgan.py
wget https://github.com/UOS-COMP6252/public/raw/main/WGAN/utils.py

The pre-trained weights are stored in a comet_ml registry. You need an api key to access it

In [ ]:
import comet_ml
from comet_ml.integration.pytorch import load_model
import torch
from torchvision.utils import make_grid
from dcgan import Generator, Discriminator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from utils import random_sample,recover_image

```comet_ml.init()``` will prompt for the api key

In [ ]:
comet_ml.init()

In [ ]:
generator=Generator(64,3,128,norm_type="GroupNorm",final_activation="tanh")
checkpoint=load_model("registry://wgan/WGAN-GP:1.0.0")
generator.load_state_dict(checkpoint["g_state_dict"])

In [ ]:
device='cuda'
generator=generator.to(device)

In [ ]:
nsamples=32
with torch.no_grad():
    noise = random_sample(256, 128, device)[:nsamples]
    fake_images = generator(noise)
grid = make_grid(fake_images, nrow=8, normalize=True)

In [ ]:
grid=recover_image(grid)
plt.imshow(grid)